In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.2 MB/s eta 0:00:00


**Step 1**: Extract taxnomic identifiers (TaxIds) of meta infomation on bacteria and archaea from Phage Host Daily database (https://phdaily.info/). The execution will take about one minute.

**Input**. the latest 'data.json' downloaded from the Download tab at the website. Or, we can use the 'data.json' downloaded on March 16th, 2024, which we used in the paper (https://zenodo.org/records/11276021). We assume the 'data.json' is located in the 'data_phd' directory.

**Output**. lineage information 'lineage.json' and name information 'scientific_names.json'

In [ ]:
import json
from os import strerror
from textwrap import fill
import pandas as pd
from collections import defaultdict
# loading the downloaded 'data.json'
with open('data_phd/data.json') as f:
    data_phd = json.load(f)

scientific_names = defaultdict(list)
lineages = {}
assemblies = {}
df = pd.DataFrame()

def fillin_taxonomydf(entry, entry_id, df, scientific_names, lineages, assemblies):
    entry_temp = entry
    entry = entry['taxonomy_ncbi']
    if entry_id in scientific_names:
        return df
    if str(entry['lineage'][-1]) != entry_id:
        print(f'Something is wrong with the lineage for {entry_id}')
        return df
    if len(df.index) % 1000 == 0:
        print(len(df.index), len(scientific_names))
    new_row = {}
    new_row['entry_taxid'] = entry_id
    new_row['entry_name'] = entry['lineage_names'][-1]
    new_row['entry_rank'] = entry['lineage_ranks'][-1]
    for taxid, name, rank in zip(reversed(entry['lineage'][:-1]), reversed(entry['lineage_names'][:-1]), reversed(entry['lineage_ranks'][:-1])):
        if name != None:
            taxid = str(taxid)
            new_row[rank] = taxid
            if not (taxid in scientific_names and name in scientific_names[taxid]):
                scientific_names[taxid].append(name)
    if not (entry_id in scientific_names and entry_id in scientific_names[entry_id]):
        scientific_names[entry_id].append(new_row['entry_name'])
        tail = []
        if entry['lineage_names'][0] == 'Viruses':
            tail = [None, 1]
        elif entry['lineage_names'][0] in ['Bacteria', 'Archaea']:
            tail = [131567, 1]
        else:
            print('{} is not supported!'.format(entry['lineage_names'][0]))
        lineages[entry_id] = list(reversed(entry['lineage'])) + tail
        if 'assemblies' in entry_temp:
            assemblies[entry_id] = entry_temp['assemblies']
    return pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

for entry in data_phd:
    taxid = entry['species_taxid']
    name = entry['name']
    if taxid in scientific_names and name in scientific_names[taxid]:
        print(f'{name} has encountered more than once')
    else:
        df = fillin_taxonomydf(entry, taxid, df, scientific_names, lineages, assemblies)

    for host in entry['hosts']:
        if 'lineage_names' in entry['hosts'][host]['taxonomy_ncbi']:
            df = fillin_taxonomydf(entry['hosts'][host], host, df, scientific_names, lineages, assemblies)
        else:
            print(f'{host} for {name} doesnot have lineage')

print(len(df.index), len(scientific_names))
cnt = 0
for taxid in scientific_names:
    if len(scientific_names[taxid]) > 1:
        cnt += 1
print(cnt)
assert cnt == 0 # sanity check for data.json
scientific_names = {k:v[0] for k,v in scientific_names.items()}
print(list(df['entry_rank'].unique())) # the bottom level taxonomic rank
print({len(lin) for lin in lineages.values()}) # the length of lineages: species, genus, faimly, order, class, phylum, superkingdom, unknown, unkwon

#with open('data_phd/scientific_names.json', 'w') as f:
#    json.dump(scientific_names, f, indent=2)
#with open('data_phd/lineages.json', 'w') as f:
#    json.dump(lineages, f, indent=2)

0 0
1000 1711
2000 3009
3000 4181
4000 5330
5000 6509
6000 7616
7000 8640
8000 9713
9000 10787
10000 11858
11000 12897
12000 13948
13000 14974
14000 15997
15000 17006
16000 18019
17000 19103
17709 19834
0
['species']
{9}


**Step 2**: Extract a URL to download complete genome for each species using biopython. Before running, please fill in your email adress so that the curators of NCBI dataabse will warn you if your exceeds a specific (downloading) limit. The execution will take about 12 minutes.

**Input**. 'lineage.json'

**Output**. 'host_urls.json'

In [ ]:
your_email = #'AAA@BBB.com'

In [ ]:
from Bio import Entrez
import json
from collections import defaultdict
Entrez.email = your_email
#with open('data_phd/lineages.json') as f:
#    lineages = json.load(f)
id_list = [key for key, lin in lineages.items() if lin[-3] != 10239] # exculde Viruses
# id_list = ['666', '562'] for example

def get_uids(id_list, assembly_uids, total_counts, assembly_urls, assembly_dics):
  # search on taxonomy database
  tax_results = Entrez.read(Entrez.epost(db="taxonomy", id=id_list))
  webenv = tax_results["WebEnv"]
  query_key = tax_results["QueryKey"]
  # link the result to assembly database
  link_results = Entrez.read(Entrez.elink(dbfrom="taxonomy", db='assembly', linkname='taxonomy_assembly',\
                                    webenv=webenv, query_key=query_key, cmd='neighbor_history'))
  webenv2 = link_results[0]['WebEnv']
  query_key2 = link_results[0]['LinkSetDbHistory'][0]['QueryKey']
  # get statistics by esummary
  import re
  batch_size = 3000
  previous_counts = batch_size
  assembly_lens = defaultdict(int)
  assembly_srcs = defaultdict(str)
  retstart = 0
  while previous_counts == batch_size:
    record = Entrez.read(Entrez.esummary(db="assembly", webenv=webenv2, query_key=query_key2, retstart=retstart, retmax=batch_size))
    previous_counts = len(record['DocumentSummarySet']['DocumentSummary'])
    for rec in record['DocumentSummarySet']['DocumentSummary']:
      if 'Taxid' in rec and 'Meta' in rec:
        taxid = rec['Taxid']
        total_len = int(re.search(r'<Stat category="total_length" sequence_tag="all">(\d+)</Stat>', rec['Meta']).groups()[0])
        ungap_len = int(re.search(r'<Stat category="ungapped_length" sequence_tag="all">(\d+)</Stat>', rec['Meta']).groups()[0])
        # extract meta info of complete genome
        if taxid in id_list and rec['AssemblyStatus'] == 'Complete Genome' and total_len == ungap_len:
          total_counts[taxid] += 1
          # RefSeq is preferred to GenBank
          if 'RsUid' in rec and len(rec['RsUid']) > 0 and len(rec['ExclFromRefSeq']) == 0:
            if assembly_lens[taxid] < total_len or assembly_srcs[taxid] != 'RefSeq':
              assembly_lens[taxid] = total_len
              assembly_srcs[taxid] = 'RefSeq'
              assembly_uids[taxid] = rec.attributes['uid']
              assembly_urls[taxid] = rec['FtpPath_RefSeq']
              assembly_dics[taxid] = rec
          elif 'GbUid' in rec and len(rec['GbUid']) > 0:
            if assembly_lens[taxid] < total_len and assembly_srcs[taxid] != 'RefSeq':
              assembly_lens[taxid] = total_len
              assembly_srcs[taxid] = 'GenBank'
              assembly_uids[taxid] = rec.attributes['uid']
              assembly_urls[taxid] = rec['FtpPath_GenBank']
              assembly_dics[taxid] = rec
    retstart += previous_counts

assembly_uids = defaultdict(str)
assembly_urls = defaultdict(str)
assembly_dics = defaultdict(dict)
total_counts = defaultdict(int)
for pos_start in range(0, len(id_list), 100):
  print(pos_start/100)
  pos_end = min(pos_start+100, len(id_list))
  get_uids(id_list[pos_start:pos_end], assembly_uids, total_counts, assembly_urls, assembly_dics)

#with open('data_phd/host_uids.json', 'w') as f:
#  json.dump(assembly_uids, f, indent=2)
#with open('data_phd/host_counts.json', 'w') as f:
#  json.dump(total_counts, f, indent=2)
#with open('data_phd/host_urls.json', 'w') as f:
#  json.dump(assembly_urls, f, indent=2)
#with open('data_phd/host_infos.json', 'w') as f:
# json.dump(assembly_dics, f, indent=2)
host_urls = assembly_urls

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0


**Step 3**: Download whole genome for each species using biopython. The execution will take about 15 minutes.

**Input**. 'host_urls.json'

**Output**. 'host.fasta' and 'host_accs.json'

In [ ]:
import urllib
import os
import gzip
import json
from collections import defaultdict
from Bio import SeqIO
def write_fasta(taxid, base_dir, writepath, host_accs):
  label = os.path.basename(base_dir)
  link = os.path.join(base_dir, label+'_genomic.fna.gz')
  data = urllib.request.urlopen(link)
  with gzip.open(data, mode='rt') as input:
    with open(writepath, 'a' if len(host_accs)>0 else 'w') as output:
      records = list(SeqIO.parse(input, "fasta"))
      SeqIO.write(records, output, 'fasta')
      for record in records:
        host_accs[taxid].append(record.id)

#with open('data_phd/host_urls.json') as f:
#    host_urls = json.load(f)
host_accs = defaultdict(list)
filename = 'data_phd/host.fasta'
for i, item in enumerate(host_urls.items()):
  taxid, url = item
  if i % 100 == 0:
    print(i)
  if len(url) == 0:
    continue
  write_fasta(taxid, url, filename, host_accs)
#with open('data_phd/host_accs.json', 'w') as f:
#  json.dump(host_accs, f, indent=2)

0
100
200
300
400
500
600
